<a href="https://colab.research.google.com/github/Sayandeep27/Reinforcement-Learning/blob/main/Q_Learning_for_Grid_World_Navigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

In [6]:
# Define the grid world environment
class GridWorld:
    def __init__(self, size=(5, 5), start=(0, 0), goal=(4, 4), obstacles=[]):
        self.size = size
        self.start = start
        self.goal = goal
        self.obstacles = obstacles
        self.state = start

    def reset(self):
        """Reset the agent's position to the starting position."""
        self.state = self.start

    def step(self, action):
        """Take a step in the environment based on the action chosen by the agent.

        Args:
        - action: The action chosen by the agent ('up', 'down', 'left', 'right').

        Returns:
        - state: The new state after taking the action.
        - reward: The reward received for the action.
        - done: Boolean indicating if the episode is done (reached the goal or obstacle).
        """
        x, y = self.state
        if action == 'up':
            new_state = (x - 1, y)
        elif action == 'down':
            new_state = (x + 1, y)
        elif action == 'left':
            new_state = (x, y - 1)
        elif action == 'right':
            new_state = (x, y + 1)
        else:
            raise ValueError("Invalid action")

        # Check if the new state is within bounds and not an obstacle
        if (0 <= new_state[0] < self.size[0] and
            0 <= new_state[1] < self.size[1] and
            new_state not in self.obstacles):
            self.state = new_state

        # Calculate reward
        if self.state == self.goal:
            reward = 1
            done = True
        else:
            reward = 0
            done = False

        return self.state, reward, done

    def visualize(self):
        """Visualize the grid world environment with positions marked."""
        grid = np.zeros(self.size)
        grid[self.start] = 0.5  # Start position
        grid[self.goal] = 0.8   # Goal position
        for obs in self.obstacles:
            grid[obs] = -1       # Obstacles
        grid[self.state] = 0.3   # Current position
        return grid


In [7]:

# Q-learning agent
class QLearningAgent:
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.1):
        """
        Initialize the Q-learning agent.

        Args:
        - env: The grid world environment object.
        - alpha: Learning rate (default: 0.1).
        - gamma: Discount factor (default: 0.9).
        - epsilon: Exploration-exploitation trade-off (default: 0.1).
        """
        self.env = env
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration-exploitation trade-off
        # Initialize Q-table with zeros, dimensions are (rows, columns, actions)
        self.q_table = np.zeros((env.size[0], env.size[1], 4))  # 4 actions: 'up', 'down', 'left', 'right'

    def choose_action(self, state):
        """
        Choose an action for the current state based on epsilon-greedy policy.

        Args:
        - state: The current state of the agent.

        Returns:
        - action: The chosen action ('up', 'down', 'left', 'right').
        """
        if np.random.uniform(0, 1) < self.epsilon:
            # Exploration: choose a random action
            return np.random.choice(['up', 'down', 'left', 'right'])
        else:
            # Exploitation: choose action with highest Q-value for this state
            return ['up', 'down', 'left', 'right'][np.argmax(self.q_table[state])]

    def update_q_table(self, state, action, reward, next_state):
        """
        Update the Q-table based on the agent's experience.

        Args:
        - state: The current state of the agent.
        - action: The action taken in the current state ('up', 'down', 'left', 'right').
        - reward: The reward received for taking the action.
        - next_state: The resulting state after taking the action.
        """
        # Retrieve the current Q-value for the state-action pair
        old_value = self.q_table[state][['up', 'down', 'left', 'right'].index(action)]
        # Calculate the maximum Q-value for the next state
        next_max = np.max(self.q_table[next_state])
        # Update the Q-value based on the Q-learning formula
        new_value = (1 - self.alpha) * old_value + self.alpha * (reward + self.gamma * next_max)
        self.q_table[state][['up', 'down', 'left', 'right'].index(action)] = new_value

    def train(self, episodes=100):
        """
        Train the Q-learning agent to learn the optimal policy.

        Args:
        - episodes: Number of episodes to train the agent (default: 100).
        """
        for _ in range(episodes):
            self.env.reset()  # Reset the environment for each episode
            state = self.env.state  # Get the initial state
            done = False  # Initialize episode completion flag
            while not done:
                action = self.choose_action(state)  # Choose action based on current state
                next_state, reward, done = self.env.step(action)  # Take action and observe next state and reward
                self.update_q_table(state, action, reward, next_state)  # Update Q-table based on experience
                state = next_state  # Move to the next state

    def test(self):
        """
        Test the trained Q-learning agent by navigating from start to goal.

        Returns:
        - path: List of states representing the optimal path found by the agent.
        """
        self.env.reset()  # Reset the environment
        state = self.env.state  # Get the initial state
        path = [state]  # Initialize path with start state
        while state != self.env.goal:
            action = ['up', 'down', 'left', 'right'][np.argmax(self.q_table[state])]  # Choose action with highest Q-value
            next_state, _, _ = self.env.step(action)  # Take action and observe next state (ignore reward and done)
            state = next_state  # Move to the next state
            path.append(state)  # Add state to the path
        return path

In [4]:
# Main function
if __name__ == "__main__":
    # Define the grid world environment
    size = (5, 5)
    start = (0, 0)
    goal = (4, 4)
    obstacles = [(1, 1), (2, 2), (3, 3)]
    env = GridWorld(size=size, start=start, goal=goal, obstacles=obstacles)

    # Initialize Q-learning agent
    agent = QLearningAgent(env)

    # Train the agent
    agent.train(episodes=100)

    # Test the agent
    path = agent.test()
    print("Optimal Path Found by Agent:", path)

    # Visualize the grid world environment
    print("Grid World Visualization:")
    print(env.visualize())

Optimal Path Found by Agent: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Grid World Visualization:
[[ 0.5  0.   0.   0.   0. ]
 [ 0.  -1.   0.   0.   0. ]
 [ 0.   0.  -1.   0.   0. ]
 [ 0.   0.   0.  -1.   0. ]
 [ 0.   0.   0.   0.   0.3]]
